# Методы отбора признаков

Это процесс отбора подмножества значимых признаков для использования в построении модели. Методы отбора применяют в случаях когда нужно сократить количество признаков для:
- ускорения алгоритмов обучения
- уменьшения потребляемой памяти
- избавления от неинформативных признаков и выбросов

Прежде, чем описывать алгоритмы, подготовим среду *jupyter notebook*.

In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

И установим внешние зависимости:

In [6]:
import numpy as np
from itertools import combinations

Также понадобятся вспомогательные методы.

1. Получает на вход выборку `dataset`. Ожидается, что `dataset` будет взят из `sklearn/datasets`, но можно любую структуру данных с аналогичным интерфейсом.

In [7]:
def get_feature_count(dataset):
    return len(dataset.data[0])

2. Получает на вход выборку `dataset` и массив прикнаков `feature`. Выбирает из выборки только признаки из `feature`, проводит по ним обучение (взял *наивного Байеса*), проводит классификацию и возвращает количество допущенных ошибок.

In [9]:
from sklearn.naive_bayes import GaussianNB

def count_error(dataset, features):
    data = dataset.data[:, features]

    gnb = GaussianNB()
    gnb.fit(data, dataset.target)

    y_predict = gnb.predict(data)

    return (dataset.target != y_predict).sum()

## Полный перебор

Полный перебор даёт 100% гарантию наилучшего ответа. Очевидно, что сложность алгоритма *экспоненциальная* и при большом количестве признаков умноженном на скорость обучения ответа можно не дождаться.

Алгоритм перебирает всевозможные наборы прикнаков и из наилучшего берет с наименьшим количеством признаков.

Введём вспомогательную функцию, которая будет по количеству признаков возвращать генераторы комбинаций прикнаков:

In [10]:
def __feat_gen(count):
    for c in range(1, count + 1):
        arr = list(range(count))
        comb = combinations(arr, c)

        yield from comb

Функция полного перебора признаков на вход получает выборку `dataset`, возвращает словарь с интерфейсом:
- `features` – массив признаков, дающий наименьшую ошибку
- `error` – ошибка

**Все последующие функции отбора признаков имеют такой же тип ответа.**

In [11]:
def selection_full_search(dataset):
    feature_count = get_feature_count(dataset)
    result = {
        "error": 9999,
        "features": []
    }

    for feat_cur in __feat_gen(feature_count):
        error = count_error(dataset, feat_cur)

        if error < result["error"]:
            result = {
                "error": error,
                "features": feat_cur
            }

    return result

**Применение и сравнение результатов будет в конце**

## Алгоритм ADD

Является жадным алгоритмом. Начинает с набора признаков `= []`. Добавляет в набор по одному наилучшему признаку до тех пор, пока качество алгоритма не начинает ухудшаться.

Второй параметр алгоритма введён только для **алгоритма ADD-DEL**. Про него можно прочитать ниже.

In [14]:
def selection_add(dataset, result=None):
    feature_count = get_feature_count(dataset)
    if result is None:
        result = {
            "error": 9999,
            "features": []
        }

    while True:
        result_cur = result

        for feat in range(feature_count):
            if feat in result["features"]:
                continue

            features = result["features"] + [feat]
            features.sort()

            error = count_error(dataset, features)
            if error <= result_cur["error"]:
                result_cur = {
                    "error": error,
                    "features": features
                }

        if result_cur == result:
            break

        result = result_cur

    return result

## Алгоритм DEL

Является жадным алгоритмом. Начинает с набора признаков = полному количеству признаков.
На каждом шаге удаляет по одному наихудшему признаку до тех пор, пока качество алгоритма не ухудшается.

Второй параметр алгоритма введён только для **алгоритма ADD-DEL**. Про него можно прочитать ниже.

In [16]:
def selection_del(dataset, result=None):
    feature_count = get_feature_count(dataset)
    if result is None:
        features_all = list(range(feature_count))
        result = {
            "error": count_error(dataset, features_all),
            "features": features_all
        }

    while len(result["features"]) > 1:
        result_cur = result

        for feat in result["features"]:
            features = result["features"][:]
            features.remove(feat)

            error = count_error(dataset, features)
            if error <= result_cur["error"]:
                result_cur = {
                    "error": error,
                    "features": features
                }

        if result_cur == result:
            break

        result = result_cur

    return result

## Алгоритм ADD-DEL

Является комбинацией **ADD** и **DEL**. Начинает с набора признаков `= []`. Далее итеративно сначала применяет **ADD** алгоритм, потом **DEL** алгоритм. Эти шаги повторяются итеративно, пока качество алгоримта растёт. Таким образом, является небольшим улучшением **ADD** алгоритма, позволяя избавить от "лишне собранных" признаков и продолжить использовать жадный метод выбора признаков.

In [17]:
def selection_add_del(dataset):
    result = {
        "error": 9999,
        "features": []
    }

    while True:
        new_result = selection_add(dataset, result)
        new_result = selection_del(dataset, new_result)

        if new_result["error"] >= result["error"]:
            break

        result = new_result

    return result

Алгоритм также можно начать и с **DEL** метода, потом применяя **ADD**. Поменяется только изначальная выборка. Она будет не пустая, а содержать все признаки.

## Сравнение Полного перебора, ADD, DEL, ADD-DEL

Сравним результаты редукции выборки разных алгоритмов на выборке `breast_cancer` по первым 12 признакам из библиотеки `sklearn`.

In [19]:
from sklearn import datasets


dataset = datasets.load_breast_cancer()
feature_count = 12
dataset.data = dataset.data[:, :feature_count]


def __log(message, result):
    print(message)
    print("Количество ошибок: ", result["error"])
    print("Количество признаков: ", len(result["features"]))
    print("Признаки: ", np.asarray(result["features"]))
    print("-------------------------------------------")


features_all = list(range(feature_count))
__log("Полный набор признаков", {
    "error": count_error(dataset, features_all),
    "features": features_all
})

__log("Полный перебор", selection_full_search(dataset))

__log("Алгоритм ADD", selection_add(dataset))

__log("Алгоритм DEL", selection_del(dataset))

__log("Алгоритм ADD-DEL", selection_add_del(dataset))

Полный набор признаков
Количество ошибок:  48
Количество признаков:  12
Признаки:  [ 0  1  2  3  4  5  6  7  8  9 10 11]
-------------------------------------------
Полный перебор
Количество ошибок:  35
Количество признаков:  5
Признаки:  [0 1 4 6 8]
-------------------------------------------
Алгоритм ADD
Количество ошибок:  36
Количество признаков:  4
Признаки:  [ 0  1  7 11]
-------------------------------------------
Алгоритм DEL
Количество ошибок:  40
Количество признаков:  3
Признаки:  [0 1 7]
-------------------------------------------
Алгоритм ADD-DEL
Количество ошибок:  36
Количество признаков:  4
Признаки:  [ 0  1  7 11]
-------------------------------------------


Как можно увидеть, **полный перебор** является лидером по наименьшей ошибке. Алгориты **ADD**, **DEL** и **ADD-DEL** дали
близкий к **полному перебору** ответ. Маленький разрыв легко объяснить небольшим количеством фичей, маленькой выбркой и
тривиальностью выборки. На практике этот разрыв будет большой. Тут важно увидеть, что жадные алгоритмы в погоне за жадностью
не увидели, что набор "менее полезных признаков" в совокупности дал наи